In [1]:
!pip install pywebhdfs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('output/metadata_cleaned.csv')

In [4]:
df.head()

,Unnamed: 0,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,...,needed_supplemental_O2,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,folder,filename,sick,age_group
0,776,407,365.0,1.0,69.0,pneumonia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,images/pneumonia,000001-11.jpg,1,old
1,639,339,240.0,0.0,64.0,pneumonia,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,images/pneumonia,000007-15.jpg,1,old
2,713,376,180.0,1.0,55.0,pneumonia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,images/pneumonia,503d2fbe68cd143b0f15749b4f816b_jumbo.jpeg,1,middle
3,528,282,120.0,1.0,46.0,pneumonia,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,images/pneumonia,000024-1.jpg,1,middle
4,537,284,98.0,1.0,52.0,pneumonia,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,images/pneumonia,000012-3.jpg,1,middle


In [5]:
df.columns.tolist()

['Unnamed: 0',
 'patientid',
 'offset',
 'sex',
 'age',
 'finding',
 'RT_PCR_positive',
 'survival',
 'intubated',
 'intubation_present',
 'went_icu',
 'in_icu',
 'needed_supplemental_O2',
 'temperature',
 'pO2_saturation',
 'leukocyte_count',
 'neutrophil_count',
 'lymphocyte_count',
 'folder',
 'filename',
 'sick',
 'age_group']

In [6]:
from pywebhdfs.webhdfs import PyWebHdfsClient

# Настройка соединения с HDFS
hdfs_client = PyWebHdfsClient(
    host='namenode',
    port='50070'
)
    
def hdfs_mkdir(new_directory_path, debug=False):    
    try:
        hdfs_client.make_dir(new_directory_path)
        if debug:
            print(f"Directory '{new_directory_path}' created successfully.")
    except Exception as e:
        print(f"Error creating directory: {e}")
        raise

def hdfs_copy_from_local(local_file_path, hdfs_file_path, debug=False):    
    try:
        with open(local_file_path, 'rb') as file_content:
            hdfs_client.create_file(hdfs_file_path, file_content.read())
        
        if debug:
            print(f"File '{local_file_path}' send successfully to HDFS!")
    except Exception as e:
        print(f"Error send file {local_file_path}: {e}")
        raise

def hdfs_rmdir(hdfs_file_path, debug=False):
    try:    
        hdfs_client.delete_file_dir(hdfs_file_path, recursive=True)    
        if debug:
            print(f"Directory '{hdfs_file_path}' deleted sucessfully.")
    except Exception as e:
        print(f"Error delete directory: {e}")

In [7]:
debug = True

# удаляем на всякий случай (после экспериментов)
hdfs_rmdir('/data/covid_dataset', debug=debug)

#создаем базовые папки
dirs = [
    '/data/covid_dataset',
    '/data/covid_dataset/images',
    '/data/covid_dataset/metadata',
    '/data/covid_dataset/processed'
]

for dir in dirs:
    hdfs_mkdir(dir, debug=debug)

Directory '/data/covid_dataset' deleted sucessfully.
Directory '/data/covid_dataset' created successfully.
Directory '/data/covid_dataset/images' created successfully.
Directory '/data/covid_dataset/metadata' created successfully.
Directory '/data/covid_dataset/processed' created successfully.


In [8]:
# создаем подпапки диагнозов
for f in df['folder'].unique():
    hdfs_mkdir('/data/covid_dataset/' + f, debug=debug)        

Directory '/data/covid_dataset/images/pneumonia' created successfully.
Directory '/data/covid_dataset/images/covid' created successfully.
Directory '/data/covid_dataset/images/no' created successfully.
Directory '/data/covid_dataset/images/tuberculosis' created successfully.
Directory '/data/covid_dataset/volumes/covid' created successfully.


In [9]:
# загружаем файлы метаданных
hdfs_copy_from_local('output/metadata_cleaned.csv','/data/covid_dataset/metadata/metadata_cleaned.csv', debug=debug)
hdfs_copy_from_local('output/metadata.parquet','/data/covid_dataset/metadata/metadata.parquet', debug=debug)

File 'output/metadata_cleaned.csv' send successfully to HDFS!
File 'output/metadata.parquet' send successfully to HDFS!


In [10]:
# раскидываем файлы по папкам
count_ok = 0
count_error = 0
for row in df.itertuples(index=True, name='Pandas'):    
    try:
        hdfs_copy_from_local('input/images/' + row.filename, '/data/covid_dataset/' + row.folder + '/' + row.filename)    
        count_ok += 1
    except Exception as e:
        count_error += 1

print(f'Finish load file: ok - {count_ok}, fail - {count_error}')

Error send file input/images/radiopaedia_org_covid-19-pneumonia-23_86359_0-dcm.nii.gz: [Errno 2] No such file or directory: 'input/images/radiopaedia_org_covid-19-pneumonia-23_86359_0-dcm.nii.gz'
Error send file input/images/radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz: [Errno 2] No such file or directory: 'input/images/radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz'
Error send file input/images/radiopaedia_org_covid-19-pneumonia-29_86491_1-dcm.nii.gz: [Errno 2] No such file or directory: 'input/images/radiopaedia_org_covid-19-pneumonia-29_86491_1-dcm.nii.gz'
Error send file input/images/radiopaedia_org_covid-19-pneumonia-27_86410_0-dcm.nii.gz: [Errno 2] No such file or directory: 'input/images/radiopaedia_org_covid-19-pneumonia-27_86410_0-dcm.nii.gz'
Error send file input/images/radiopaedia_org_covid-19-pneumonia-29_86490_1-dcm.nii.gz: [Errno 2] No such file or directory: 'input/images/radiopaedia_org_covid-19-pneumonia-29_86490_1-dcm.nii.gz'
Error send file input/